In [19]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import cv2
from pycocotools.coco import COCO
from PIL import Image
import torch
import os
import matplotlib.pyplot as plt
#import colormap as cmap
import numpy as np

def horisontal_flip(images, targets):
    images = torch.flip(images, [-1])
    targets[2] = 1 - targets[2]
    return images, targets

def visualize(index, masks, bounding_boxes, augment):
    image = coco.loadImgs(coco.getImgIds()[index])[0]    
    anno_ids = coco.getAnnIds(imgIds=[image["id"]])
    annos = coco.loadAnns(anno_ids) 
    #print( annos )
    """new_annos = []
    for single in annos:
        single = get_angles(single)
        new_annos.append([single])
    
    annos = new_annos"""
    filename = os.path.join(data_folder,image["file_name"])
    #print(filename)
    img = cv2.imread(str(filename))
    print(img.shape)
            
    for anno in annos:
        #print(anno)
        anno = get_angles(anno)
        if augment:
            if np.random.random() < 0.5:
                box = anno['bbox']
                img, anno = horisontal_flip(torch.Tensor(img), box)
                anno['bbox'] = anno
        
        #c = tuple([int(x) for x in anno["category_id"]])         
        if masks:
            points = np.array(anno["segmentation"], dtype=np.int32).reshape(1,-1,2)
            img = cv2.polylines(img, points, True, 3)
        
        if bounding_boxes:  
                x,y,w,h,a = np.array(anno["bbox"]).astype(np.int32)
                #img = cv2.rectangle(img, (x,y),(x + w,y + h),(0,255,255), 3) 
                img = draw_xywha(img,x,y,w,h,a)
                cat = coco.cats[anno['category_id']]['name']
                cv2.putText(img, cat, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX ,  0.8, 1, cv2.LINE_AA)            
                
    
    plt.imshow(img[...,::-1])
    plt.axis("off")
    plt.grid(False)
    plt.tight_layout()

def get_angles(ann):
    
    #print(ann['bbox'])
    img_id = ann['image_id']
    new_ann = None
    # get width and height
    if not 'rbbox' in ann:
        # using COCO dataset. 4 = [x1,y1,w,h]
        coco = True
        # convert COCO format: x1,y1,w,h to x,y,w,h
        ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
        ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
        ann['bbox'].append(0)
        if ann['bbox'][2] > ann['bbox'][3]:
            ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
            ann['bbox'][4] -= 90
        new_ann = ann['bbox']
    else:
        # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
        assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
        new_ann = ann['rbbox']
        

        if new_ann[2] > new_ann[3]:
            new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
            new_ann[4] -= 90 if new_ann[4] > 0 else -90

    if new_ann[2] == new_ann[3]:
        new_ann[3] += 1 # force that w < h
        
    new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

    assert new_ann[2] < new_ann[3]
    assert new_ann[4] >= -90 and new_ann[4] < 90

    # override original bounding box with rotated bounding box
    ann['bbox'] = torch.Tensor(new_ann)
    #print(ann['bbox'])
    return ann

    
def draw_xywha(im, x, y, w, h, angle, color=(255,0,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w/2, -h/2], [w/2, -h/2], [w/2, h/2], [-w/2, h/2]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([x, y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    return im
    
    
data_folder = "/localdata/saurabh/dataset/FES/JPEGImages/"
coco = COCO("/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json")
#data_folder = "/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/JPEGImages/"
#coco = COCO("/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/instances_training.json")
plt.figure(figsize=(100,100))
total = len(coco.getImgIds()) - 1

interact(visualize, index=widgets.IntSlider(min=0, max=total, step=1, continuous_update=False), masks=False, bounding_boxes=True, augment=False);  
plt.show()

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


<Figure size 7200x7200 with 0 Axes>

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=300), Checkbox(valu…

# Make the data files

In [15]:
######## NOTE #######
'''Last performed for FES dataset. The logic differes slightly for other datasets. 
Take care of image extensions and image_names variable during implementation.
Logic of load annotation is verified. 
Do visualize the targets before training to check if correct annotations are saved'''
####### NOTE ######
import glob
from collections import defaultdict
import os
import json
import numpy as np
import cv2
from PIL import Image
import torch

###Input####
mode_in = 'person'   ### ['train', 'val']
dataset = 'fes'
image_paths = glob.glob(f"/localdata/saurabh/yolov3/data/{dataset}/images/{mode_in}/*.jpg")
#image_paths = glob.glob('/localdata/saurabh/dataset/DST/val/*.png')   ### for DST dataset
############


image_ids = []
image_names = []
#print(image_paths[0])
##### Edit this part according to the name of the files ####
for image in image_paths:
    #print(image)
    image_name, image_ext = os.path.splitext(os.path.basename(image))
    #print(image_name)
    image_names += [image_name]
    image_name = image_name.split('_')[1]     # Varies according to databases. Check if name is correct wrt to anno
    image_ids += [image_name]     # If image_ids != image_names, execute the function below to get sequential image ids
    #break
    
#print(image_ids)

In [16]:
len(image_ids)

301

In [17]:
image_names

['Record_00893',
 'Record_00766',
 'Record_00650',
 'Record_00888',
 'Record_00643',
 'Record_00626',
 'Record_00855',
 'Record_00826',
 'Record_00655',
 'Record_00702',
 'Record_00614',
 'Record_00880',
 'Record_00886',
 'Record_00700',
 'Record_00768',
 'Record_00734',
 'Record_00755',
 'Record_00781',
 'Record_00799',
 'Record_00629',
 'Record_00793',
 'Record_00750',
 'Record_00637',
 'Record_00672',
 'Record_00805',
 'Record_00696',
 'Record_00803',
 'Record_00691',
 'Record_00794',
 'Record_00809',
 'Record_00679',
 'Record_00812',
 'Record_00724',
 'Record_00640',
 'Record_00675',
 'Record_00890',
 'Record_00699',
 'Record_00858',
 'Record_00622',
 'Record_00804',
 'Record_00673',
 'Record_00879',
 'Record_00857',
 'Record_00795',
 'Record_00670',
 'Record_00881',
 'Record_00872',
 'Record_00852',
 'Record_00618',
 'Record_00848',
 'Record_00694',
 'Record_00732',
 'Record_00769',
 'Record_00633',
 'Record_00634',
 'Record_00635',
 'Record_00802',
 'Record_00692',
 'Record_00824

In [4]:
#### Process and load annotatinos according to RAPID implementation
def load_anns(img_dir,json_path):
        coco = False
        print(f'Loading annotations {json_path} into memory...')
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        for ann in json_data['annotations']:
            img_id = ann['image_id']
            #print(img_id)
            new_ann = None
            # get width and height 
            if not 'rbbox' in ann:
                # using COCO dataset. 4 = [x1,y1,w,h]
                coco = True
                ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
                ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
                ann['bbox'].append(0)
                if ann['bbox'][2] > ann['bbox'][3]:
                    ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
                    ann['bbox'][4] -= 90
                new_ann = ann['bbox']
            else:
                # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
                assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
                new_ann = ann['rbbox']


                if new_ann[2] > new_ann[3]:
                    new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
                    new_ann[4] -= 90 if new_ann[4] > 0 else -90

            if new_ann[2] == new_ann[3]:
                new_ann[3] += 1 # force that w < h

            new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

            assert new_ann[2] < new_ann[3]
            assert new_ann[4] >= -90 and new_ann[4] < 90

            # override original bounding box with rotated bounding box
            ann['bbox'] = torch.Tensor(new_ann)
            #print(img_id)
            imgid2anns[img_id].append(ann)

        for img in json_data['images']:
            img_id = img['id']
            #print(img_id)
            assert img_id not in imgid2path
            anns = imgid2anns[img_id]
            # if there is crowd gt, skip this image
            if coco and any(ann['iscrowd'] for ann in anns):
                continue

            img_ids.append(img_id)
            imgid2path[img_id] = os.path.join(img_dir, img['file_name'])
            imgid2info[img['id']] = img

        catids = [cat['id'] for cat in json_data['categories']]

In [5]:
### Store the required values generated during load_anns function
coco =  False
img_ids = []
imgid2info = dict()
imgid2path = dict()
imgid2anns = defaultdict(list)
catids = []

if dataset == 'DST':
    load_anns("/localdata/saurabh/dataset/DST/val/", "/localdata/saurabh/dataset/DST/instances_val_with_rbbox.json" )
elif dataset == 'custom':
    load_anns("/localdata/saurabh/dataset/theodore_v3/images/", "/localdata/saurabh/dataset/theodore_v3/coco/annotations/instances.json")
elif dataset == 'fes':
    load_anns("/localdata/saurabh/dataset/fes/val/", "/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json" )

Loading annotations /localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json into memory...


In [6]:
img_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [7]:
#### Run this to get image_ids in sequence starting with 1 #####
'''Do not run if img_id=image_name'''

image_ids = []
for i in range(len(imgid2info)):
    file_name = imgid2info[i+1]['file_name'].split('.')[0]
    #i_d = imgid2info
    
    if file_name in image_names:  # Make sure names in both list are identical
        image_ids += [i+1]
        print(file_name, i+1)
    

Record_00600 1
Record_00601 2
Record_00602 3
Record_00603 4
Record_00604 5
Record_00605 6
Record_00606 7
Record_00607 8
Record_00608 9
Record_00609 10
Record_00610 11
Record_00611 12
Record_00612 13
Record_00613 14
Record_00614 15
Record_00615 16
Record_00616 17
Record_00617 18
Record_00618 19
Record_00619 20
Record_00620 21
Record_00621 22
Record_00622 23
Record_00623 24
Record_00624 25
Record_00625 26
Record_00626 27
Record_00627 28
Record_00628 29
Record_00629 30
Record_00630 31
Record_00631 32
Record_00632 33
Record_00633 34
Record_00634 35
Record_00635 36
Record_00636 37
Record_00637 38
Record_00638 39
Record_00639 40
Record_00640 41
Record_00641 42
Record_00642 43
Record_00643 44
Record_00644 45
Record_00645 46
Record_00646 47
Record_00647 48
Record_00648 49
Record_00649 50
Record_00650 51
Record_00651 52
Record_00652 53
Record_00653 54
Record_00654 55
Record_00655 56
Record_00656 57
Record_00657 58
Record_00658 59
Record_00659 60
Record_00660 61
Record_00661 62
Record_00662 63
R

In [8]:
image_names = sorted(image_names) #if image_ids are sequential then sort the names accordingly

In [9]:
for a in zip(image_names, image_ids):
    print(a)

('Record_00600', 1)
('Record_00601', 2)
('Record_00602', 3)
('Record_00603', 4)
('Record_00604', 5)
('Record_00605', 6)
('Record_00606', 7)
('Record_00607', 8)
('Record_00608', 9)
('Record_00609', 10)
('Record_00610', 11)
('Record_00611', 12)
('Record_00612', 13)
('Record_00613', 14)
('Record_00614', 15)
('Record_00615', 16)
('Record_00616', 17)
('Record_00617', 18)
('Record_00618', 19)
('Record_00619', 20)
('Record_00620', 21)
('Record_00621', 22)
('Record_00622', 23)
('Record_00623', 24)
('Record_00624', 25)
('Record_00625', 26)
('Record_00626', 27)
('Record_00627', 28)
('Record_00628', 29)
('Record_00629', 30)
('Record_00630', 31)
('Record_00631', 32)
('Record_00632', 33)
('Record_00633', 34)
('Record_00634', 35)
('Record_00635', 36)
('Record_00636', 37)
('Record_00637', 38)
('Record_00638', 39)
('Record_00639', 40)
('Record_00640', 41)
('Record_00641', 42)
('Record_00642', 43)
('Record_00643', 44)
('Record_00644', 45)
('Record_00645', 46)
('Record_00646', 47)
('Record_00647', 48)
(

In [10]:
imgid2info[105] # just to cross check the image_id-image_name pair

{'id': 105,
 'width': 1680,
 'height': 1680,
 'file_name': 'Record_00704.jpg',
 'license': 0,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': 0}

In [11]:
#### Get annotations only for the train/val set according to image_ids
ind_path = [imgid2path[int(ids)] for ids in image_ids]
annotations = [imgid2anns[int(ids)] for ids in image_ids]

In [12]:
##### Map the labels to 0-indexed labels
label_map = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5}

In [13]:
label_map[3]

2

In [14]:
##### Save the paths to the images and annotations as .txt files

all_images = []
text_names = open(f'/localdata/saurabh/yolov3/data/{dataset}/{mode_in}_paths.txt', "w")
for i, ann in enumerate(annotations):
    file_name = image_names[i]     # image_names should be in sync with annotation sequence
    #### Append the path list
    text_names.write(f'/localdata/saurabh/yolov3/data/{dataset}/images/{mode_in}/{file_name}{image_ext}')
    text_names.write('\n')
    #### Open the label file for each anno
    text_file = open(f'/localdata/saurabh/yolov3/data/{dataset}/labels/{mode_in}/{file_name}.txt', "w") 
    for anno in ann:
        #print(anno['category_id'])
        if anno['category_id'] == 1:   ### save annotation only for person class
            text_file.write("%i " %label_map[anno['category_id']])
            text_file.writelines(["%f " % item  for item in anno['bbox']])
            text_file.write("\n")
    
    text_file.close()
    #break
text_names.close()

print(file_name)
    


Record_00900


In [22]:
x = np.array([0,1,0.02, 0.1], dtype='float16')

In [25]:
print(np.array(x, dtype='uint8'))

[0 1 0 0]
